# UFoE Phase 5b-2: GCN4 ColabFold Multimer

**목적**: ESMFold pseudo-dimer의 한계를 넘어 진짜 multimer 구조 예측

| Group | 이름 | 서열 (33aa) | 곤% | 건% |
|-------|------|-------------|-----|-----|
| A | GCN4_WT | RMKQLEDKVEELLSKNYHLENEVARLKKLVGER | 33% | 45% |
| B | GCN4_GON | LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL | 48% | 30% |
| C | GCN4_SALT | REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR | 21% | 67% |

**3 Analyses**:
1. Inter-chain salt bridge (e-g' registry)
2. Core Cb-Cb packing (a,d positions)
3. Crossing angle (18-25 expected)

**2026-02-13 Young Kang + Claude**

---
## Cell 1: ColabFold 설치

ColabFold + AlphaFold2-multimer 설치 (~5분 소요)

In [ ]:
%%time
import os

# ColabFold 설치
if not os.path.isfile("COLABFOLD_READY"):
    print("[1/4] ColabFold 설치 중...")
    os.system("pip install -q \"colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold\"")
    
    print("[2/4] JAX GPU 설치 중...")
    os.system("pip install -q \"jax[cuda12]\"")

    print("[3/4] Biopython 설치 중...")
    os.system("pip install -q biopython")

    print("[4/4] 설치 확인...")
    os.system("touch COLABFOLD_READY")
    print("\n=== ColabFold 설치 완료! ===")
    print("⚠️ 런타임이 재시작될 수 있습니다. 재시작 후 이 셀부터 다시 실행하세요.")
else:
    print("ColabFold already installed. Skipping.")

---
## Cell 2: 설정 + 확정 서열

In [ ]:
import os
import json
import numpy as np
from pathlib import Path
from collections import Counter, defaultdict

# ═══════════════════════════════════════════════════
# 곤감리건 7-3-6-4 매핑
# ═══════════════════════════════════════════════════
AA_TO_TYPE = {
    'A': 'gon', 'L': 'gon', 'V': 'gon', 'I': 'gon',
    'F': 'gon', 'M': 'gon', 'W': 'gon',
    'G': 'gam', 'C': 'gam', 'P': 'gam',
    'S': 'ri', 'T': 'ri', 'N': 'ri', 'Q': 'ri', 'H': 'ri', 'Y': 'ri',
    'E': 'geon', 'D': 'geon', 'K': 'geon', 'R': 'geon',
}

MAX_ASA = {
    'ALA': 129, 'ARG': 274, 'ASN': 195, 'ASP': 193, 'CYS': 167,
    'GLN': 225, 'GLU': 223, 'GLY': 104, 'HIS': 224, 'ILE': 197,
    'LEU': 201, 'LYS': 236, 'MET': 224, 'PHE': 240, 'PRO': 159,
    'SER': 155, 'THR': 172, 'TRP': 285, 'TYR': 263, 'VAL': 174,
}

# Heptad register: defgabcdefgabcdefgabcdefgabcdefga
HEPTAD = 'defgabcdefgabcdefgabcdefgabcdefga'
A_POS = [i for i, h in enumerate(HEPTAD) if h == 'a']  # 4,11,18,25,32
D_POS = [i for i, h in enumerate(HEPTAD) if h == 'd']  # 0,7,14,21,28
E_POS = [i for i, h in enumerate(HEPTAD) if h == 'e']  # 1,8,15,22,29
G_POS = [i for i, h in enumerate(HEPTAD) if h == 'g']  # 3,10,17,24,31
CORE_POS = sorted(A_POS + D_POS)

# ═══════════════════════════════════════════════════
# 확정 서열 (3 Groups)
# ═══════════════════════════════════════════════════
GROUPS = {
    'A': {
        'name': 'GCN4_WT',
        'seq': 'RMKQLEDKVEELLSKNYHLENEVARLKKLVGER',
        'desc': 'Wild-type GCN4 (PDB 2ZTA)',
    },
    'B': {
        'name': 'GCN4_GON',
        'seq': 'LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL',
        'desc': '곤-강화: a,d = L/I 100% hydrophobic core',
    },
    'C': {
        'name': 'GCN4_SALT',
        'seq': 'REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR',
        'desc': '건-강화: e=E, g=K 100% salt bridge max',
    },
}

# 디렉토리 생성
OUTPUT_DIR = Path('ufoe_5b2_output')
PDB_DIR = OUTPUT_DIR / 'pdb'
FASTA_DIR = OUTPUT_DIR / 'fasta'
for d in [OUTPUT_DIR, PDB_DIR, FASTA_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ═══════════════════════════════════════════════════
# 서열 확인
# ═══════════════════════════════════════════════════
print('=' * 70)
print('  UFoE Phase 5b-2: GCN4 확정 서열')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    g = GROUPS[gname]
    seq = g['seq']
    types = [AA_TO_TYPE[aa] for aa in seq]
    c = Counter(types)
    n = len(seq)

    core_aa = [seq[i] for i in CORE_POS if i < n]
    core_gon = sum(1 for aa in core_aa if AA_TO_TYPE[aa] == 'gon')

    eg_aa = [seq[i] for i in sorted(E_POS + G_POS) if i < n]
    eg_geon = sum(1 for aa in eg_aa if AA_TO_TYPE[aa] == 'geon')

    print(f"  Group {gname}: {g['desc']}")
    print(f"  [{g['name']}] {seq} ({n}aa)")
    print(f"    곤={c['gon']/n:.0%} 감={c['gam']/n:.0%} 리={c['ri']/n:.0%} 건={c['geon']/n:.0%}")
    print(f"    a,d core: {''.join(core_aa)} (곤 {core_gon/len(core_aa)*100:.0f}%)")
    print(f"    e,g salt: {''.join(eg_aa)} (건 {eg_geon/len(eg_aa)*100:.0f}%)")
    print()

---
## Cell 3: ColabFold Multimer 예측 실행

각 그룹의 homodimer를 AlphaFold2-multimer-v3로 예측 (~10-15분/그룹)

In [ ]:
%%time
import subprocess

# FASTA 파일 생성 (ColabFold multimer 형식: seq:seq)
for gname in ['A', 'B', 'C']:
    g = GROUPS[gname]
    seq = g['seq']
    fasta_path = FASTA_DIR / f"{g['name']}.fasta"
    with open(fasta_path, 'w') as f:
        f.write(f">{g['name']}\n{seq}:{seq}\n")
    print(f"  {gname}: {fasta_path}")

# 배치 FASTA
batch_fasta = FASTA_DIR / 'gcn4_5b2_all.fasta'
with open(batch_fasta, 'w') as f:
    for gname in ['A', 'B', 'C']:
        g = GROUPS[gname]
        f.write(f">{g['name']}\n{g['seq']}:{g['seq']}\n")

print(f"\n  배치 FASTA: {batch_fasta}")
print(f"\n{'=' * 70}")
print(f"  ColabFold Multimer 예측 시작")
print(f"{'=' * 70}\n")

# ColabFold 배치 실행
result_dir = OUTPUT_DIR / 'colabfold_results'
result_dir.mkdir(parents=True, exist_ok=True)

cmd = [
    'colabfold_batch',
    str(batch_fasta),
    str(result_dir),
    '--num-models', '5',
    '--num-recycle', '3',
    '--model-type', 'alphafold2_multimer_v3',
    '--pair-mode', 'unpaired_paired',
]

print(f"  명령: {' '.join(cmd)}")
print(f"  예상 소요: ~30-45분 (3 그룹 × 5 모델)")
print(f"  결과 디렉토리: {result_dir}")
print()

process = subprocess.run(cmd, capture_output=False, text=True)

if process.returncode == 0:
    print("\n\n=== ColabFold 예측 완료! ===")
else:
    print(f"\n\n⚠️ ColabFold 오류 발생 (returncode={process.returncode})")
    print("아래의 개별 실행 셀을 사용해 보세요.")

### (대안) 개별 그룹 실행

배치 실행에 문제가 있을 경우, 그룹별로 개별 실행:

In [ ]:
# 개별 그룹 실행 (필요시)
# 실행할 그룹을 선택하세요: 'A', 'B', 또는 'C'
RUN_GROUP = 'A'  # <-- 변경하여 실행

g = GROUPS[RUN_GROUP]
fasta_path = FASTA_DIR / f"{g['name']}.fasta"
group_result_dir = OUTPUT_DIR / 'colabfold_results'
group_result_dir.mkdir(parents=True, exist_ok=True)

print(f"Group {RUN_GROUP} ({g['name']}) 개별 실행")
print(f"서열: {g['seq']}")
print(f"FASTA: {fasta_path}")
print()

!colabfold_batch {str(fasta_path)} {str(group_result_dir)} \
    --num-models 5 --num-recycle 3 \
    --model-type alphafold2_multimer_v3 \
    --pair-mode unpaired_paired

---
## Cell 4: 결과 PDB 파일 찾기

ColabFold 출력에서 best-ranked PDB 파일 자동 탐색

In [ ]:
import glob

result_dir = OUTPUT_DIR / 'colabfold_results'

# ColabFold 출력 구조: {name}_relaxed_rank_001_*.pdb 또는 {name}_unrelaxed_rank_001_*.pdb
pdb_files = {}

for gname in ['A', 'B', 'C']:
    g = GROUPS[gname]
    name = g['name']

    # rank_001 (best) PDB 찾기 — relaxed 우선, 없으면 unrelaxed
    patterns = [
        str(result_dir / f"{name}_relaxed_rank_001_*.pdb"),
        str(result_dir / f"{name}_unrelaxed_rank_001_*.pdb"),
        str(result_dir / f"{name}*rank_001*.pdb"),
        str(result_dir / f"{name}*rank001*.pdb"),
        str(result_dir / f"{name}*.pdb"),
    ]

    found = None
    for pat in patterns:
        matches = sorted(glob.glob(pat))
        if matches:
            found = matches[0]
            break

    if found:
        pdb_files[gname] = found
        print(f"  Group {gname} ({name}): {found}")
    else:
        print(f"  Group {gname} ({name}): ⚠️ PDB not found")
        # 사용자가 수동 경로 지정 가능
        # pdb_files[gname] = '/path/to/your/pdb'

print(f"\n  PDB 파일 {len(pdb_files)}/3 발견")

if len(pdb_files) < 3:
    print("\n  ⚠️ 누락된 PDB 파일이 있습니다.")
    print("  결과 디렉토리 내용:")
    for f in sorted(glob.glob(str(result_dir / '*'))):
        print(f"    {f}")

### (수동 경로 지정)

자동 탐색이 실패한 경우, 아래 셀에서 직접 PDB 경로를 지정하세요:

In [ ]:
# 필요한 경우만 실행: 수동으로 PDB 경로 지정
# pdb_files['A'] = 'ufoe_5b2_output/colabfold_results/GCN4_WT_relaxed_rank_001_model_1.pdb'
# pdb_files['B'] = 'ufoe_5b2_output/colabfold_results/GCN4_GON_relaxed_rank_001_model_1.pdb'
# pdb_files['C'] = 'ufoe_5b2_output/colabfold_results/GCN4_SALT_relaxed_rank_001_model_1.pdb'

print("현재 PDB 경로:")
for gname in ['A', 'B', 'C']:
    path = pdb_files.get(gname, 'NOT SET')
    exists = os.path.exists(path) if path != 'NOT SET' else False
    print(f"  Group {gname}: {path} {'✅' if exists else '❌'}")

---
## Cell 5: pLDDT + pTM 점수 확인

ColabFold JSON 결과에서 confidence 점수 추출

In [ ]:
from Bio.PDB import PDBParser

parser = PDBParser(QUIET=True)
confidence_data = {}

print('=' * 70)
print('  pLDDT + Chain별 점수')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    if gname not in pdb_files:
        print(f"  Group {gname}: SKIP (no PDB)")
        continue

    pdb_path = pdb_files[gname]
    g = GROUPS[gname]

    structure = parser.get_structure(gname, pdb_path)
    chains = list(structure[0].get_chains())

    chain_plddts = {}
    all_plddts = []

    for chain in chains:
        chain_id = chain.id
        plddts = []
        for res in chain.get_residues():
            for atom in res:
                if atom.name == 'CA':
                    plddts.append(atom.bfactor)
                    break

        if plddts:
            # bfactor가 0-1 범위면 100x
            if max(plddts) <= 1.0:
                plddts = [p * 100 for p in plddts]
            chain_plddts[chain_id] = round(np.mean(plddts), 1)
            all_plddts.extend(plddts)

    total_plddt = round(np.mean(all_plddts), 1) if all_plddts else 0

    confidence_data[gname] = {
        'chains': chain_plddts,
        'total': total_plddt,
        'n_chains': len(chains),
    }

    print(f"  Group {gname} ({g['name']}): {len(chains)} chains")
    for cid, plddt in chain_plddts.items():
        print(f"    Chain {cid}: pLDDT = {plddt}")
    print(f"    Total pLDDT = {total_plddt}")

    # JSON 점수 파일도 확인
    json_pattern = pdb_path.replace('.pdb', '').replace('_relaxed', '').replace('_unrelaxed', '')
    for jf in glob.glob(str(result_dir / f"{g['name']}*scores*.json")):
        with open(jf) as f:
            scores = json.load(f)
        if 'ptm' in scores:
            print(f"    pTM = {scores['ptm']:.3f}")
        if 'iptm' in scores:
            print(f"    ipTM = {scores['iptm']:.3f}")
        if 'ptm' in scores and 'iptm' in scores:
            ranking = 0.8 * scores['iptm'] + 0.2 * scores['ptm']
            print(f"    Ranking score (0.8*ipTM + 0.2*pTM) = {ranking:.3f}")
            confidence_data[gname]['ptm'] = round(scores['ptm'], 3)
            confidence_data[gname]['iptm'] = round(scores['iptm'], 3)
            confidence_data[gname]['ranking'] = round(ranking, 3)
    print()

---
## Cell 6: 분석 1 — Inter-chain Salt Bridge 검출

ColabFold 진짜 multimer에서 e-g' registry salt bridge 검출

In [ ]:
from Bio.PDB import PDBParser

def detect_salt_bridges_multimer(pdb_file, seq, cutoff=4.0):
    """ColabFold multimer PDB에서 inter-chain salt bridge 검출.

    ColabFold multimer: 별도 chain A, B로 출력됨.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('s', pdb_file)
    chains = list(structure[0].get_chains())

    if len(chains) < 2:
        print(f"    ⚠️ Chain 2개 미만: {len(chains)}개")
        return {'all': [], 'total': 0, 'canonical': [], 'canonical_count': 0}

    chain_a_res = list(chains[0].get_residues())
    chain_b_res = list(chains[1].get_residues())

    neg_atoms_map = {'GLU': ['OE1', 'OE2'], 'ASP': ['OD1', 'OD2']}
    pos_atoms_map = {'LYS': ['NZ'], 'ARG': ['NH1', 'NH2']}

    bridges = []

    def check_pair(res_i, i_idx, res_j, j_idx, direction):
        rn_i = res_i.get_resname()
        rn_j = res_j.get_resname()

        # neg(i) ↔ pos(j)
        neg_atoms = neg_atoms_map.get(rn_i, [])
        pos_atoms = pos_atoms_map.get(rn_j, [])

        for na in neg_atoms:
            for pa in pos_atoms:
                if na in res_i and pa in res_j:
                    d = res_i[na] - res_j[pa]
                    if d < cutoff:
                        h_i = HEPTAD[i_idx] if i_idx < len(HEPTAD) else '?'
                        h_j = HEPTAD[j_idx] if j_idx < len(HEPTAD) else '?'
                        aa_i = seq[i_idx] if i_idx < len(seq) else '?'
                        aa_j = seq[j_idx] if j_idx < len(seq) else '?'
                        canonical = (h_i in ('e', 'g') and h_j in ('e', 'g'))
                        bridges.append({
                            'chain_a_pos': i_idx + 1 if direction == 'AB' else j_idx + 1,
                            'chain_b_pos': j_idx + 1 if direction == 'AB' else i_idx + 1,
                            'aa_a': aa_i if direction == 'AB' else aa_j,
                            'aa_b': aa_j if direction == 'AB' else aa_i,
                            'heptad_a': h_i if direction == 'AB' else h_j,
                            'heptad_b': h_j if direction == 'AB' else h_i,
                            'atoms': f"{na}-{pa}" if direction == 'AB' else f"{pa}-{na}",
                            'dist': round(d, 2),
                            'canonical_eg': canonical,
                            'direction': direction,
                        })

    # A(neg) → B(pos)
    for i, res_a in enumerate(chain_a_res):
        for j, res_b in enumerate(chain_b_res):
            check_pair(res_a, i, res_b, j, 'AB')

    # B(neg) → A(pos)
    for j, res_b in enumerate(chain_b_res):
        for i, res_a in enumerate(chain_a_res):
            check_pair(res_b, j, res_a, i, 'BA')

    # 중복 제거 (같은 잔기 쌍 → 가장 가까운 것만)
    unique = {}
    for b in bridges:
        key = (b['chain_a_pos'], b['chain_b_pos'])
        rev_key = (b['chain_b_pos'], b['chain_a_pos'])
        if key not in unique or b['dist'] < unique[key]['dist']:
            unique[key] = b
        if rev_key not in unique or b['dist'] < unique[rev_key]['dist']:
            unique[rev_key] = b

    result = sorted(unique.values(), key=lambda x: x['dist'])
    # 진짜 유일한 쌍만 남기기
    seen = set()
    final = []
    for b in result:
        pair = tuple(sorted([b['chain_a_pos'], b['chain_b_pos']]))
        if pair not in seen:
            seen.add(pair)
            final.append(b)

    canonical = [b for b in final if b['canonical_eg']]

    return {
        'all': final,
        'total': len(final),
        'canonical': canonical,
        'canonical_count': len(canonical),
    }


# ═══════════════════════════════════════════════════
# 실행
# ═══════════════════════════════════════════════════
salt_bridge_results = {}

print('=' * 70)
print('  분석 1: Inter-chain Salt Bridge')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    if gname not in pdb_files:
        print(f"  Group {gname}: SKIP")
        continue

    g = GROUPS[gname]
    print(f"  ── Group {gname}: {g['name']} ──")

    sb = detect_salt_bridges_multimer(pdb_files[gname], g['seq'])
    salt_bridge_results[gname] = sb

    print(f"    Total: {sb['total']}")
    print(f"    Canonical (e-g'): {sb['canonical_count']}")

    for b in sb['all'][:10]:
        tag = ' ★ canonical' if b['canonical_eg'] else ''
        print(f"      {b['aa_a']}{b['chain_a_pos']}({b['heptad_a']}) — "
              f"{b['aa_b']}{b['chain_b_pos']}({b['heptad_b']}) "
              f"dist={b['dist']}Å [{b['atoms']}]{tag}")
    print()

---
## Cell 7: 분석 2 — Core Cb-Cb Packing (a,d positions)

In [ ]:
def analyze_core_packing_multimer(pdb_file, seq):
    """ColabFold multimer PDB에서 a,d 위치 inter-chain Cb-Cb 거리.

    코일드코일 정상: a<->a' 6-8A, d<->d' 6-8A, a<->d' 5-7A.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('s', pdb_file)
    chains = list(structure[0].get_chains())

    if len(chains) < 2:
        return {'error': f'Need 2 chains, got {len(chains)}'}

    res_a = list(chains[0].get_residues())
    res_b = list(chains[1].get_residues())

    def get_cb(res):
        if 'CB' in res:
            return res['CB'].get_vector().get_array()
        if 'CA' in res:
            return res['CA'].get_vector().get_array()
        return None

    distances = {'a_a': [], 'd_d': [], 'a_d': [], 'd_a': []}
    n_a = min(len(seq), len(res_a))
    n_b = min(len(seq), len(res_b))

    # a(chain A) vs a(chain B)
    for ia in A_POS:
        if ia >= n_a:
            continue
        cb_a = get_cb(res_a[ia])
        if cb_a is None:
            continue
        for ib in A_POS:
            if ib >= n_b:
                continue
            cb_b = get_cb(res_b[ib])
            if cb_b is not None:
                distances['a_a'].append(np.linalg.norm(np.array(cb_a) - np.array(cb_b)))

    # d(chain A) vs d(chain B)
    for ia in D_POS:
        if ia >= n_a:
            continue
        cb_a = get_cb(res_a[ia])
        if cb_a is None:
            continue
        for ib in D_POS:
            if ib >= n_b:
                continue
            cb_b = get_cb(res_b[ib])
            if cb_b is not None:
                distances['d_d'].append(np.linalg.norm(np.array(cb_a) - np.array(cb_b)))

    # a(A) vs d(B), d(A) vs a(B)
    for ia in A_POS:
        if ia >= n_a:
            continue
        cb_a = get_cb(res_a[ia])
        if cb_a is None:
            continue
        for ib in D_POS:
            if ib >= n_b:
                continue
            cb_b = get_cb(res_b[ib])
            if cb_b is not None:
                distances['a_d'].append(np.linalg.norm(np.array(cb_a) - np.array(cb_b)))

    for ia in D_POS:
        if ia >= n_a:
            continue
        cb_a = get_cb(res_a[ia])
        if cb_a is None:
            continue
        for ib in A_POS:
            if ib >= n_b:
                continue
            cb_b = get_cb(res_b[ib])
            if cb_b is not None:
                distances['d_a'].append(np.linalg.norm(np.array(cb_a) - np.array(cb_b)))

    stats = {}
    for key, vals in distances.items():
        if vals:
            stats[key] = {
                'min': round(min(vals), 2),
                'mean': round(np.mean(vals), 2),
                'std': round(np.std(vals), 2),
                'n': len(vals),
            }
        else:
            stats[key] = None

    # Same-heptad 위치 비교 (a_i(A) vs a_i(B), d_i(A) vs d_i(B))
    same_heptad = []
    for idx, pos in enumerate(A_POS):
        if pos >= n_a or pos >= n_b:
            continue
        cb_a = get_cb(res_a[pos])
        cb_b = get_cb(res_b[pos])
        if cb_a is not None and cb_b is not None:
            d = np.linalg.norm(np.array(cb_a) - np.array(cb_b))
            same_heptad.append({
                'type': 'a', 'heptad_idx': idx, 'pos': pos + 1,
                'aa_a': seq[pos], 'aa_b': seq[pos], 'dist': round(d, 2),
            })

    for idx, pos in enumerate(D_POS):
        if pos >= n_a or pos >= n_b:
            continue
        cb_a = get_cb(res_a[pos])
        cb_b = get_cb(res_b[pos])
        if cb_a is not None and cb_b is not None:
            d = np.linalg.norm(np.array(cb_a) - np.array(cb_b))
            same_heptad.append({
                'type': 'd', 'heptad_idx': idx, 'pos': pos + 1,
                'aa_a': seq[pos], 'aa_b': seq[pos], 'dist': round(d, 2),
            })

    return {'all_distances': stats, 'same_heptad': same_heptad}


# ═══════════════════════════════════════════════════
# 실행
# ═══════════════════════════════════════════════════
core_packing_results = {}

print('=' * 70)
print('  분석 2: Core Cb-Cb Packing (a,d positions)')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    if gname not in pdb_files:
        continue

    g = GROUPS[gname]
    print(f"  ── Group {gname}: {g['name']} ──")

    cp = analyze_core_packing_multimer(pdb_files[gname], g['seq'])
    core_packing_results[gname] = cp

    if 'error' in cp:
        print(f"    Error: {cp['error']}")
        continue

    for key in ['a_a', 'd_d', 'a_d']:
        s = cp['all_distances'].get(key)
        if s:
            print(f"    {key}: min={s['min']}A mean={s['mean']}A std={s['std']}A")

    print(f"    Same-heptad (closest 5):")
    for sh in sorted(cp['same_heptad'], key=lambda x: x['dist'])[:5]:
        print(f"      {sh['type']}{sh['pos']}: {sh['aa_a']}↔{sh['aa_b']} = {sh['dist']}A")
    print()

---
## Cell 8: 분석 3 — Crossing Angle + Inter-helix Distance

In [ ]:
def calc_crossing_angle_multimer(pdb_file, seq):
    """PCA로 두 chain의 helix 축 사이 crossing angle 계산.

    GCN4 coiled-coil 기대값: 18-25 degrees, parallel.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('s', pdb_file)
    chains = list(structure[0].get_chains())

    if len(chains) < 2:
        return {'error': f'Need 2 chains, got {len(chains)}'}

    # CA 좌표 수집
    def get_ca_coords(chain):
        coords = []
        for res in chain.get_residues():
            if 'CA' in res:
                coords.append(res['CA'].get_vector().get_array())
        return np.array(coords)

    ca_a = get_ca_coords(chains[0])
    ca_b = get_ca_coords(chains[1])

    if len(ca_a) < 5 or len(ca_b) < 5:
        return {'error': f'Not enough CA atoms: {len(ca_a)}, {len(ca_b)}'}

    def helix_axis(coords):
        center = np.mean(coords, axis=0)
        centered = coords - center
        cov = np.cov(centered.T)
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        axis = eigenvectors[:, np.argmax(eigenvalues)]
        return axis, center

    axis_a, center_a = helix_axis(ca_a)
    axis_b, center_b = helix_axis(ca_b)

    # Crossing angle
    cos_angle = np.dot(axis_a, axis_b) / (np.linalg.norm(axis_a) * np.linalg.norm(axis_b))
    cos_angle = max(-1, min(1, cos_angle))
    crossing = np.degrees(np.arccos(abs(cos_angle)))

    is_parallel = cos_angle > 0
    inter_dist = np.linalg.norm(center_a - center_b)

    return {
        'crossing_angle': round(crossing, 1),
        'is_parallel': is_parallel,
        'inter_helix_dist': round(inter_dist, 1),
        'coiled_coil': crossing < 30 and is_parallel,
    }


# ═══════════════════════════════════════════════════
# 실행
# ═══════════════════════════════════════════════════
crossing_angle_results = {}

print('=' * 70)
print('  분석 3: Crossing Angle + Inter-helix Distance')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    if gname not in pdb_files:
        continue

    g = GROUPS[gname]
    print(f"  ── Group {gname}: {g['name']} ──")

    ca = calc_crossing_angle_multimer(pdb_files[gname], g['seq'])
    crossing_angle_results[gname] = ca

    if 'error' in ca:
        print(f"    Error: {ca['error']}")
    else:
        cc_ok = '✅' if ca['coiled_coil'] else '❌'
        par = '✅ parallel' if ca['is_parallel'] else '❌ anti-parallel'
        in_range = '✅' if 18 <= ca['crossing_angle'] <= 25 else '⚠️'
        print(f"    Crossing angle: {ca['crossing_angle']}° {in_range}")
        print(f"    Direction: {par}")
        print(f"    Inter-helix dist: {ca['inter_helix_dist']}A")
        print(f"    Coiled-coil: {cc_ok}")
    print()

---
## Cell 9: 보너스 — SASA Burial Ratio

In [ ]:
from Bio.PDB import PDBParser, ShrakeRupley

def calc_burial_ratio_multimer(pdb_file, seq):
    """a,d core vs surface SASA 비율 (ColabFold multimer).

    burial_ratio < 0.6 = 코어 매몰 성공
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('s', pdb_file)
    model = structure[0]

    # 전체 complex SASA 계산
    sr = ShrakeRupley()
    sr.compute(model, level='R')

    # Chain A의 잔기별 SASA
    chains = list(model.get_chains())
    if not chains:
        return {'error': 'No chains'}

    chain_a_res = list(chains[0].get_residues())
    n = min(len(seq), len(chain_a_res))

    residue_rsa = []
    for i in range(n):
        res = chain_a_res[i]
        rn = res.get_resname()
        ma = MAX_ASA.get(rn, 200)
        residue_rsa.append(res.sasa / ma)

    core_vals = [residue_rsa[i] for i in CORE_POS if i < n]
    surf_vals = [residue_rsa[i] for i in range(n) if i not in CORE_POS]

    if not core_vals or not surf_vals:
        return {'error': 'Not enough residues'}

    core_mean = np.mean(core_vals)
    surf_mean = np.mean(surf_vals)
    ratio = core_mean / surf_mean if surf_mean > 0 else 999

    # 곤감리건별 SASA
    type_sasa = defaultdict(list)
    for i in range(n):
        aa = seq[i]
        t = AA_TO_TYPE.get(aa, '?')
        type_sasa[t].append(residue_rsa[i])

    type_means = {t: round(np.mean(v), 4) for t, v in type_sasa.items() if v}

    return {
        'core_mean': round(core_mean, 4),
        'surface_mean': round(surf_mean, 4),
        'burial_ratio': round(ratio, 4),
        'burial_ok': ratio < 0.6,
        'type_means': type_means,
    }


# ═══════════════════════════════════════════════════
# 실행
# ═══════════════════════════════════════════════════
burial_results = {}

print('=' * 70)
print('  보너스: SASA Burial Ratio')
print('=' * 70)
print()

for gname in ['A', 'B', 'C']:
    if gname not in pdb_files:
        continue

    g = GROUPS[gname]
    print(f"  ── Group {gname}: {g['name']} ──")

    br = calc_burial_ratio_multimer(pdb_files[gname], g['seq'])
    burial_results[gname] = br

    if 'error' in br:
        print(f"    Error: {br['error']}")
    else:
        ok = '✅' if br['burial_ok'] else '❌'
        print(f"    Core SASA (a,d): {br['core_mean']:.4f}")
        print(f"    Surface SASA: {br['surface_mean']:.4f}")
        print(f"    burial_ratio: {br['burial_ratio']:.4f} {ok}")
        print(f"    곤감리건별 SASA: {br['type_means']}")
    print()

---
## Cell 10: 종합 비교표 + 판정

In [ ]:
print('=' * 80)
print('  UFoE Phase 5b-2: ColabFold Multimer 종합 결과')
print('=' * 80)
print()

# ── 비교표 ──
header = f"{'측정':25s} | {'A (WT)':15s} | {'B (곤강화)':15s} | {'C (건강화)':15s}"
print(f"  {header}")
print(f"  {'-' * 78}")

# pLDDT
row = f"{'pLDDT (total)':25s}"
for gn in ['A', 'B', 'C']:
    cd = confidence_data.get(gn, {})
    v = f"{cd.get('total', 'N/A')}"
    row += f" | {v:15s}"
print(f"  {row}")

# ipTM (ColabFold only)
row = f"{'ipTM':25s}"
for gn in ['A', 'B', 'C']:
    cd = confidence_data.get(gn, {})
    v = f"{cd.get('iptm', 'N/A')}"
    row += f" | {v:15s}"
print(f"  {row}")

# Salt bridge total
row = f"{'Salt bridge (total)':25s}"
for gn in ['A', 'B', 'C']:
    sb = salt_bridge_results.get(gn, {})
    v = f"{sb.get('total', 'N/A')}"
    row += f" | {v:15s}"
print(f"  {row}")

# Salt bridge canonical
row = f"{'Salt bridge (e-g\')':25s}"
for gn in ['A', 'B', 'C']:
    sb = salt_bridge_results.get(gn, {})
    v = f"{sb.get('canonical_count', 'N/A')}"
    row += f" | {v:15s}"
print(f"  {row}")

# Core packing
for key_label, key in [('a↔a\' Cb min (A)', 'a_a'), ('d↔d\' Cb min (A)', 'd_d')]:
    row = f"{key_label:25s}"
    for gn in ['A', 'B', 'C']:
        cp = core_packing_results.get(gn, {})
        ad = cp.get('all_distances', {}).get(key) if isinstance(cp, dict) and 'error' not in cp else None
        v = f"{ad['min']:.1f}" if ad else 'N/A'
        row += f" | {v:15s}"
    print(f"  {row}")

# Crossing angle
row = f"{'Crossing angle (deg)':25s}"
for gn in ['A', 'B', 'C']:
    ca = crossing_angle_results.get(gn, {})
    v = f"{ca.get('crossing_angle', 'N/A')}" if 'error' not in ca else 'N/A'
    row += f" | {v:15s}"
print(f"  {row}")

# Inter-helix distance
row = f"{'Inter-helix dist (A)':25s}"
for gn in ['A', 'B', 'C']:
    ca = crossing_angle_results.get(gn, {})
    v = f"{ca.get('inter_helix_dist', 'N/A')}" if 'error' not in ca else 'N/A'
    row += f" | {v:15s}"
print(f"  {row}")

# Burial ratio
row = f"{'burial_ratio':25s}"
for gn in ['A', 'B', 'C']:
    br = burial_results.get(gn, {})
    v = f"{br.get('burial_ratio', 'N/A'):.3f}" if isinstance(br.get('burial_ratio'), (int, float)) else 'N/A'
    row += f" | {v:15s}"
print(f"  {row}")

# ── ESMFold 비교 ──
print(f"\n  {'=' * 78}")
print(f"  ESMFold pseudo-dimer vs ColabFold Multimer 비교")
print(f"  {'=' * 78}")

esmfold_ref = {
    'A': {'plddt': 69.0, 'salt_total': 0, 'salt_eg': 0, 'dd_min': 7.24, 'angle': 24.8, 'burial': 0.529},
    'B': {'plddt': 80.9, 'salt_total': 0, 'salt_eg': 0, 'dd_min': 4.82, 'angle': 20.2, 'burial': 0.535},
    'C': {'plddt': 71.7, 'salt_total': 5, 'salt_eg': 1, 'dd_min': 15.53, 'angle': 11.7, 'burial': 1.345},
}

print(f"\n  {'측정':25s} | {'ESMFold':15s} | {'ColabFold':15s} | 변화")
print(f"  {'-' * 78}")

for gn in ['A', 'B', 'C']:
    g = GROUPS[gn]
    print(f"\n  == Group {gn}: {g['name']} ==")
    esm = esmfold_ref[gn]

    cd = confidence_data.get(gn, {})
    sb = salt_bridge_results.get(gn, {})
    cp = core_packing_results.get(gn, {})
    ca = crossing_angle_results.get(gn, {})
    br = burial_results.get(gn, {})

    cf_plddt = cd.get('total', 'N/A')
    print(f"  {'pLDDT':25s} | {esm['plddt']:15.1f} | {str(cf_plddt):15s} |")

    cf_salt = sb.get('total', 'N/A')
    print(f"  {'Salt bridge (total)':25s} | {esm['salt_total']:15d} | {str(cf_salt):15s} |")

    cf_eg = sb.get('canonical_count', 'N/A')
    print(f"  {'Salt bridge (e-g\')':25s} | {esm['salt_eg']:15d} | {str(cf_eg):15s} |")

    dd = cp.get('all_distances', {}).get('d_d') if isinstance(cp, dict) and 'error' not in cp else None
    cf_dd = f"{dd['min']:.2f}" if dd else 'N/A'
    print(f"  {'d↔d\' Cb min':25s} | {esm['dd_min']:15.2f} | {str(cf_dd):15s} |")

    cf_angle = ca.get('crossing_angle', 'N/A') if 'error' not in ca else 'N/A'
    print(f"  {'Crossing angle':25s} | {esm['angle']:15.1f} | {str(cf_angle):15s} |")

    cf_burial = f"{br['burial_ratio']:.3f}" if isinstance(br.get('burial_ratio'), (int, float)) else 'N/A'
    print(f"  {'burial_ratio':25s} | {esm['burial']:15.3f} | {str(cf_burial):15s} |")

---
## Cell 11: 3가지 원리 판정

In [ ]:
print('=' * 70)
print('  Phase 5b-2 판정: 3가지 원리')
print('=' * 70)
print()

judgments = {}

# ── 1. 곤 코어 원리 ──
# B의 d↔d' packing이 A보다 밀착해야 함
print('  [1] 곤 코어 원리: B의 core packing ≤ A?')
print('      (곤 100% at a,d → 코어 더 밀착)')

cp_a = core_packing_results.get('A', {})
cp_b = core_packing_results.get('B', {})

if isinstance(cp_a, dict) and 'error' not in cp_a and isinstance(cp_b, dict) and 'error' not in cp_b:
    dd_a = cp_a.get('all_distances', {}).get('d_d', {})
    dd_b = cp_b.get('all_distances', {}).get('d_d', {})

    if dd_a and dd_b:
        a_min = dd_a.get('min', 999)
        b_min = dd_b.get('min', 999)
        ok = b_min <= a_min
        judgments['gon_core'] = ok
        print(f'      A d↔d\' min = {a_min:.2f}A')
        print(f'      B d↔d\' min = {b_min:.2f}A')
        print(f'      → {"✅ PASS" if ok else "❌ FAIL"}: B {"<=" if ok else ">"} A')
    else:
        print('      → ⚠️ 데이터 부족')
else:
    print('      → ⚠️ 데이터 부족')

print()

# ── 2. 건 표면 원리 ──
# C에서 salt bridge ≥ A, 동시에 burial_ratio > A (코어 붕괴)
print('  [2] 건 표면 원리: C salt bridge ≥ A + C burial > A?')
print('      (건 100% at e,g → bridge 형성 but 코어 붕괴)')

sb_a = salt_bridge_results.get('A', {})
sb_c = salt_bridge_results.get('C', {})
br_a = burial_results.get('A', {})
br_c = burial_results.get('C', {})

a_salt = sb_a.get('total', 0)
c_salt = sb_c.get('total', 0)
a_burial = br_a.get('burial_ratio', 999)
c_burial = br_c.get('burial_ratio', 0)

salt_ok = c_salt >= a_salt
burial_ok = c_burial > a_burial
both_ok = salt_ok and burial_ok
judgments['geon_surface'] = both_ok

print(f'      Salt: A={a_salt} vs C={c_salt} → {"✅" if salt_ok else "❌"}')
print(f'      Burial: A={a_burial:.3f} vs C={c_burial:.3f} → {"✅" if burial_ok else "❌"}')
print(f'      → {"✅ PASS" if both_ok else "❌ FAIL"}')

# Canonical salt bridge 상세
if sb_a.get('canonical'):
    print(f'      A canonical e-g\' bridges ({sb_a["canonical_count"]}):')
    for b in sb_a['canonical'][:5]:
        print(f'        {b["aa_a"]}{b["chain_a_pos"]}({b["heptad_a"]}) — '
              f'{b["aa_b"]}{b["chain_b_pos"]}({b["heptad_b"]}) {b["dist"]}A')

print()

# ── 3. 균형 설계 조건 ──
# WT crossing angle이 18-25 범위 내
print('  [3] 균형 설계 조건: WT crossing angle 18-25 deg?')
print('      (곤 코어 + 건 표면 = 최적 균형)')

ca_a = crossing_angle_results.get('A', {})
if 'error' not in ca_a:
    angle = ca_a.get('crossing_angle', 999)
    ok = 18 <= angle <= 25
    judgments['balance'] = ok
    print(f'      WT crossing angle = {angle}°')
    print(f'      18-25° range: {"✅ PASS" if ok else "⚠️ CHECK"}')
    
    # B, C도 비교
    ca_b = crossing_angle_results.get('B', {})
    ca_c = crossing_angle_results.get('C', {})
    if 'error' not in ca_b:
        print(f'      B crossing angle = {ca_b.get("crossing_angle", "N/A")}°')
    if 'error' not in ca_c:
        print(f'      C crossing angle = {ca_c.get("crossing_angle", "N/A")}°')
else:
    print(f'      → ⚠️ Error: {ca_a.get("error")}')

# ── 최종 판정 ──
print(f"\n{'=' * 70}")
print("  최종 판정")
print(f"{'=' * 70}")
print()

pass_count = sum(1 for v in judgments.values() if v)
total_count = len(judgments)

for name, ok in judgments.items():
    label = {
        'gon_core': '곤 코어 원리',
        'geon_surface': '건 표면 원리',
        'balance': '균형 설계 조건',
    }.get(name, name)
    print(f"  {label}: {'✅ PASS' if ok else '❌ FAIL'}")

print(f"\n  결과: {pass_count}/{total_count} PASS")

if pass_count == total_count:
    print("\n  🎉 ColabFold Multimer로 곤감리건 3원리 모두 검증 성공!")
else:
    print(f"\n  ⚠️ {total_count - pass_count}개 원리 추가 검증 필요")

---
## Cell 12: 결과 저장 + 다운로드

In [ ]:
# 전체 결과 JSON 저장
final_results = {
    'experiment': 'UFoE Phase 5b-2 ColabFold Multimer',
    'date': '2026-02-13',
    'groups': {},
    'judgments': judgments,
}

for gname in ['A', 'B', 'C']:
    g = GROUPS[gname]
    final_results['groups'][gname] = {
        'name': g['name'],
        'seq': g['seq'],
        'desc': g['desc'],
        'confidence': confidence_data.get(gname, {}),
        'salt_bridges': salt_bridge_results.get(gname, {}),
        'core_packing': core_packing_results.get(gname, {}),
        'crossing_angle': crossing_angle_results.get(gname, {}),
        'burial': burial_results.get(gname, {}),
    }

# ESMFold 비교 데이터
final_results['esmfold_comparison'] = {
    'A': {'plddt': 69.0, 'salt_total': 0, 'salt_eg': 0, 'dd_min': 7.24, 'angle': 24.8, 'burial': 0.529},
    'B': {'plddt': 80.9, 'salt_total': 0, 'salt_eg': 0, 'dd_min': 4.82, 'angle': 20.2, 'burial': 0.535},
    'C': {'plddt': 71.7, 'salt_total': 5, 'salt_eg': 1, 'dd_min': 15.53, 'angle': 11.7, 'burial': 1.345},
}

# 저장
result_json_path = OUTPUT_DIR / 'phase5b2_colabfold_results.json'
with open(result_json_path, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2, default=str)

print(f"  결과 저장: {result_json_path}")

# Google Colab 다운로드
try:
    from google.colab import files
    files.download(str(result_json_path))
    print("\n  📥 JSON 파일 다운로드 시작")

    # PDB 파일도 다운로드
    for gname in ['A', 'B', 'C']:
        if gname in pdb_files and os.path.exists(pdb_files[gname]):
            files.download(pdb_files[gname])
            print(f"  📥 Group {gname} PDB 다운로드")
except ImportError:
    print("\n  (Google Colab 아님 — 수동으로 파일을 다운로드하세요)")
    print(f"  JSON: {result_json_path}")
    for gname in ['A', 'B', 'C']:
        if gname in pdb_files:
            print(f"  PDB {gname}: {pdb_files[gname]}")

---
## Cell 13: 구조 시각화 (Optional)

py3Dmol로 3D 구조 확인

In [ ]:
try:
    import py3Dmol
except ImportError:
    os.system('pip install -q py3Dmol')
    import py3Dmol

def visualize_dimer(pdb_path, title=''):
    """코일드-코일 다이머 3D 시각화."""
    with open(pdb_path) as f:
        pdb_text = f.read()

    view = py3Dmol.view(width=600, height=400)
    view.addModel(pdb_text, 'pdb')

    # Chain A: cyan, Chain B: orange
    view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'cyan'}})
    view.setStyle({'chain': 'B'}, {'cartoon': {'color': 'orange'}})

    # a,d core residues: spheres
    for pos in CORE_POS:
        resi = pos + 1  # 1-indexed
        view.addStyle({'chain': 'A', 'resi': resi}, {'stick': {'color': 'blue'}})
        view.addStyle({'chain': 'B', 'resi': resi}, {'stick': {'color': 'red'}})

    # e,g salt bridge positions: thin sticks
    for pos in sorted(E_POS + G_POS):
        resi = pos + 1
        view.addStyle({'chain': 'A', 'resi': resi}, {'stick': {'color': 'green', 'radius': 0.1}})
        view.addStyle({'chain': 'B', 'resi': resi}, {'stick': {'color': 'yellow', 'radius': 0.1}})

    view.zoomTo()
    if title:
        print(f"\n  {title}")
    return view


# 각 그룹 시각화
for gname in ['A', 'B', 'C']:
    if gname in pdb_files and os.path.exists(pdb_files[gname]):
        g = GROUPS[gname]
        view = visualize_dimer(
            pdb_files[gname],
            f"Group {gname}: {g['name']} ({g['desc']})"
        )
        view.show()

---

## 실행 가이드

### Google Colab 실행 순서:
1. **Cell 1**: ColabFold 설치 (~5분)
2. **Cell 2**: 설정 + 서열 확인
3. **Cell 3**: ColabFold Multimer 예측 (~30-45분)
4. **Cell 4**: PDB 파일 탐색
5. **Cell 5-9**: 분석 실행 (순서대로)
6. **Cell 10**: 종합 비교표
7. **Cell 11**: 3원리 판정
8. **Cell 12**: 결과 저장 + 다운로드

### 주의사항:
- GPU 런타임 필수 (런타임 > 런타임 유형 변경 > T4 GPU)
- ColabFold 설치 후 런타임 재시작 필요할 수 있음
- 3 그룹 × 5 모델 = 15 예측 → ~30-45분 소요
- 무료 Colab은 시간 제한이 있으므로, 개별 그룹 실행 옵션 사용 가능

### ESMFold 기준선 (Phase 5b-2 결과):

| 측정 | A (WT) | B (곤강화) | C (건강화) |
|------|--------|-----------|----------|
| pLDDT | 69.0 | **80.9** | 71.7 |
| Salt bridge (e-g') | 0 | 0 | 1 |
| d↔d' Cb min | 7.24A | **4.82A** | 15.53A |
| Crossing angle | **24.8 deg** | **20.2 deg** | 11.7 deg |
| burial_ratio | 0.529 | 0.535 | 1.345 |

### 성공 기준:
- 곤 코어 원리: B d↔d' ≤ A d↔d' ✅
- 건 표면 원리: C salt ≥ A + C burial > A ✅  
- 균형 설계: WT angle 18-25 deg ✅